# F0 and F1 Estimation

## F0 Estimate via Ideal F0 Estimation
The ideal F0 Estimation algorithm estimates the number of distinct elements in a data stream/set (F0)

## Data
Data stream is simulated by generating a numpy array via the function "generateSeq()". 

## Problem statement

Implement the idealized F0 estimation algorithm.

A random hash function h:U -> [0,1] will be used for this implementation. Notice that this hash function takes a number and hashes it to a value from range 0 to 1. 

3 different types of hashes will be used:
1. 2-wise independent hash family
2. 4-wise independent hash family
3. murmuhash.hash(x, signed = False)

For each type of hash, 1,000 trials will be run where each trial will produce its own estimate of F0. Afterwards, you will average the 1,000 estimates of F0 to produce a final estimate. 

In [2]:
import numpy as np
import math
import random
random.seed(None)
from tqdm import tqdm
import mmh3

LARGE_PRIME = 157
n_trials = 1000

def generateSeq():
    arr=np.random.randint(0,100, size=(5000))
    return arr
    
def generateHashCoefficients(k,p):
    '''
    Input:
        k: int, represents the k-wise independent hash family
        p: int, large prime number given in "FINAL_LARGE_PRIME"
    Output:
        List of length k which represents the coefficients of k-length polynomial from x^0 to x^(k-1)
        
    generates the coefficients of a k-wise independent hash family represented by a polynomial function
    
    Each coefficient is drawn UAR from [p]
    Examples:
        24x^2 + 8x + 4 is represented as the list [4, 8, 24]
        77x + 12 is repsented as the list [12, 77]

    '''
    
    coeff = np.random.randint(0, high=p, size=k)
    return coeff
        
def kWiseHash(coeffs, p, x):
    '''
    Input:
        coeffs: List of length k, which is generated by "generateHashCoefficients" function
        p: int, large prime number given in "FINAL_LARGE_PRIME"
        x: value to be hashed
        
    Output:
        value between 0 and 1 which is the hash(x)/p
    
    hashes the value, x, based on the polynomial coefficients generated from "generatehashCoefficients", and the prime number p
    '''
    
    val = 0
    for i in range(len(coeffs)):
        val += coeffs[i]*x^i
    
    val = np.mod(val, p)
    return val / p
    
def kWiseHashFMAlg(k):
    '''
    Input:
        k = int, represents k-wise independent hash family. For this problem k = 2 and k = 4
        
    Output:
        None

    Print out the average F0 estimation accross 1000 trials for k-wise independent hash family

    Note, for each trial, the polynomial coefficients should be regenerated using "generateHashCoefficients".
    But within a single trial, the same hash function should be applied to all data points
    '''
    trials = []
    inpData=(generateSeq())
    for i in tqdm(range(n_trials)):
        V = float('inf')
        
        coeff = generateHashCoefficients(k, LARGE_PRIME)
        for j in range(len(inpData)):
            hashX = kWiseHash(coeff, LARGE_PRIME, inpData[j])
            if hashX < V:
                V = hashX
        if V > 0:
            trials.append((1/V) - 1)
            
    print('K-Wise Hash Estimate:', np.mean(trials))
    return

def murmurHashFMAlg():
    '''
    Input: None
    Output: None
    
    Print out the average F0 estimation accross 1000 trials for ideal F0 estimation using murmurHash
    '''
    
    trials = []
    inpData=(generateSeq())
    print('len', len(inpData))
    for i in tqdm(range(n_trials)):
        V = float('inf')
        
        for j in range(len(inpData)):
            hashX = mmh3.hash(inpData[j], signed=False)
            hashX /= (2**32)
        
            if hashX < V:
                V = hashX
        if V > 0:
            trials.append((1/V)-1)
    
    print('MMH3 Hash Estimate:', np.mean(trials))
    return

#your code for testing below
print('2-Wise Hash Estimate')
kWiseHashFMAlg(2)
print('--')
print('4-Wise Hash Estimate')
kWiseHashFMAlg(4)
print('--')
murmurHashFMAlg()

  1%|          | 6/1000 [00:00<00:18, 54.34it/s]

2-Wise Hash Estimate


  1%|          | 6/1000 [00:00<00:17, 55.97it/s]

K-Wise Hash Estimate: 156.0
--
4-Wise Hash Estimate


  4%|▎         | 36/1000 [00:00<00:02, 348.61it/s]

K-Wise Hash Estimate: 99.72925363693865
--
len 5000


100%|██████████| 1000/1000 [00:03<00:00, 310.46it/s]

MMH3 Hash Estimate: 465.07059411811156


## F1 Estimate via Morris Algorithm
The Morris algorithm estimates the number of elements in a data stream (F1).

## Data
The same generateSeq() function from F0 estimation is copied below, and will be used in this problem as well

## Problem Statement

MorrisCount algorithm will be repeated 1,000 times while recording the estimation of each trial. Estimations will then be averaged for the final estimation. 

In [37]:
import numpy as np
import math
#from tqdm import tqdm

def generateSeq():
    arr=np.random.randint(0,9999, size=(10000))
    return arr

def MorrisAlgo(stream):
    
    c = 0
    for i in range(len(stream)):
        if np.random.binomial(1, 0.5**c) == 1:
            c += 1
    
    return 2**c - 1

def MorrisRpt(nTrials,stream):
    
    estimates = []
    for i in range(nTrials):
        estimates.append(MorrisAlgo(stream))
        
    return np.mean(estimates)

seq = generateSeq()
print('Number of elements:', MorrisRpt(1000, seq))


Number of elements: 9892.888
